In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Well, I don't know if this is a good thing or if I did it right, but it's a start.

The technique is to add values to the test data in order to create a larger training dataset.

For more information about the method, I recommend reading:
[https://www.kaggle.com/cdeotte/pseudo-labeling-qda-0-969](http://)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Train a model and generate labels from tests

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")

In [ ]:
X = train.drop(["row_id","target"], axis = 1)
y = train['target']

In [ ]:
Xtr, Xval, ytr, yval = train_test_split(X, y, train_size = 0.8)

model = ExtraTreesClassifier(n_estimators = 800, random_state = 0, n_jobs = -1)
model.fit(Xtr, ytr)

y_pred = model.predict(Xval) 
accuracy = accuracy_score(yval, y_pred) 
print("Accuracy: %.6f" % (accuracy))

In [ ]:
pred = model.predict(test.drop("row_id", axis = 1))

## Add the generated labels on the test data, concat the train and test and repeat the training with the new dataset

In [ ]:
test['target'] = pred
full_data = pd.concat([train, test])

In [ ]:
X = full_data.drop(["row_id","target"], axis = 1)
y = full_data['target']

In [ ]:
Xtr, Xval, ytr, yval = train_test_split(X, y, train_size = 0.8)

model = ExtraTreesClassifier(n_estimators = 800, random_state = 0, n_jobs = -1)
model.fit(Xtr, ytr)

y_pred = model.predict(Xval) 
accuracy = accuracy_score(yval, y_pred) 
print("Accuracy: %.6f%%" % (accuracy))

## Submission

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")
submission.head()

In [ ]:
predicts = model.predict(test.drop(["row_id","target"], axis = 1))

submission['target'] = predicts
submission.head()

In [ ]:
submission.to_csv("submission_labels.csv", index = False)

In [ ]:
nan